# Day 17 - 2026/01/28

## 题目列表
- 148. Sort List (Medium)
- 23. Merge K Sorted Lists (Hard)
- 25. Reverse Nodes in k-Group (Hard)
- 138. Copy List with Random Pointer (Medium)
- 146. LRU Cache (Medium)

## ⏱ 预计用时: 3-4小时

In [2]:
from typing import List, Optional

class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

def build_linked_list(nums: List[int]) -> Optional[ListNode]:
    dummy = ListNode()
    current = dummy
    for num in nums:
        current.next = ListNode(num)
        current = current.next
    return dummy.next

def linked_list_to_list(head: Optional[ListNode]) -> List[int]:
    result = []
    while head:
        result.append(head.val)
        head = head.next
    return result

def build_cycle_list(nums: List[int], pos: int) -> Optional[ListNode]:
    if not nums:
        return None
    head = build_linked_list(nums)
    if pos < 0:
        return head
    tail = head
    cycle_entry = None
    idx = 0
    while tail.next:
        if idx == pos:
            cycle_entry = tail
        tail = tail.next
        idx += 1
    if idx == pos:
        cycle_entry = tail
    if cycle_entry:
        tail.next = cycle_entry
    return head

class RandomNode:
    def __init__(self, x: int, next: 'RandomNode' = None, random: 'RandomNode' = None):
        self.val = int(x)
        self.next = next
        self.random = random

def build_random_list(data: List[tuple]) -> Optional[RandomNode]:
    if not data:
        return None
    nodes = [RandomNode(val) for val, _ in data]
    for i in range(len(nodes) - 1):
        nodes[i].next = nodes[i + 1]
    for i, (_, rand_i) in enumerate(data):
        if rand_i is not None:
            nodes[i].random = nodes[rand_i]
    return nodes[0]

## 148. Sort List (Medium)

Given the head of a linked list, return the list after sorting it in ascending order.

**Example:**
- Input: head = [4,2,1,3]
- Output: [1,2,3,4]


In [ ]:
# slow - fast操作，既可以寻找环，又可以寻找中点或特定位置点
# 中点断链，记得存prev指针，因为slow跑完会刚好在[0:mid]和[mid:]中间，mid是在后半区间的，要断就断prev
# 断链操作：pointer.next = None
# 注意每次pointer的更新啊啊啊啊！ cur = cur.next

class Solution:
    def sortList(self, head: Optional[ListNode]) -> Optional[ListNode]:
        # TODO: Implement your solution
        if not head or not head.next: return head
        # 找中点
        prev = None
        slow = fast = head
        while fast and fast.next:
            prev = slow
            slow = slow.next
            fast = fast.next.next
        prev.next = None
        newa = self.sortList(head)
        newb = self.sortList(slow)
        return self.merge(newa, newb)

    def merge(self, heada, headb):
        dummy = ListNode()
        cur = dummy
        while heada and headb:
            if heada.val <= headb.val:
                cur.next = heada
                heada = heada.next
            else:
                cur.next = headb
                headb = headb.next
            cur = cur.next
        while heada:
            cur.next = heada
            heada = heada.next
            cur = cur.next
        while headb:
            cur.next = headb
            headb = headb.next
            cur = cur.next
        return dummy.next

# Test cases
def test_sortList():
    s = Solution()
    head = build_linked_list([1,2,3,4])
    result = s.sortList(head)
    assert linked_list_to_list(result) == [1,2,3,4]
    head = build_linked_list([-1,5,3,4,0])
    result = s.sortList(head)
    assert linked_list_to_list(result) == [-1,0,3,4,5]
    assert linked_list_to_list(s.sortList(None)) == []
    print("All test cases passed!")

test_sortList()


All test cases passed!


## 23. Merge K Sorted Lists (Hard)

You are given an array of k linked-lists lists, each linked-list is sorted in ascending order.
Merge all the linked-lists into one sorted linked-list and return it.

**Example:**
- Input: lists = [[1,4,5],[1,3,4],[2,6]]
- Output: [1,1,2,3,4,4,5,6]


In [13]:
# 和上面的sortlist一样，这道题也是拿寻常的做法套在特殊数据结构上
# 整体思路很简单，就是heapify，但链表如何被heap才是难点
# 看看需要存的是哪些信息：（val，idx，node本身）

'''
heapq 默认以元组第一个元素为主、第二个为次依次比较，所以 (val, idx, node) 已经是你想要的。
只要用 heapq.heappop(heap) 就会自动按 val、idx 排序弹出最小值节点
'''
# 合并K个链表，就是维护表头，用heap每次压出，挨个串联起来
# 只存储头部即可，每次压进压出一个，作为frontier的更新
# 这道题适合练习heap的语法

import heapq

class Solution:
    def mergeKLists(self, lists: List[Optional[ListNode]]) -> Optional[ListNode]:
        # TODO: Implement your solution
        # 维护一个堆节点的head
        heap = []
        for i, nums in enumerate(lists):
            if nums:
                heapq.heappush(heap, (nums.val, i, nums))
        dummy = ListNode()
        cur = dummy
        while heap:
            temp = heapq.heappop(heap)
            if temp[-1].next:
                heapq.heappush(heap, (temp[-1].next.val, temp[1], temp[-1].next))
            cur.next = temp[-1]
            cur = cur.next
        return dummy.next

# Test cases
def test_mergeKLists():
    s = Solution()
    lists = [build_linked_list(x) for x in [[1,4,5],[1,3,4],[2,6]]]
    result = s.mergeKLists(lists)
    assert linked_list_to_list(result) == [1,1,2,3,4,4,5,6]
    assert linked_list_to_list(s.mergeKLists([])) == []
    assert linked_list_to_list(s.mergeKLists([None])) == []
    print("All test cases passed!")

test_mergeKLists()


All test cases passed!


## 25. Reverse Nodes in k-Group (Hard)

Given the head of a linked list, reverse the nodes of the list k at a time, and return the modified list.

**Example:**
- Input: head = [1,2,3,4,5], k = 2
- Output: [2,1,4,3,5]


In [3]:
class Solution:
    def reverseKGroup(self, head: Optional[ListNode], k: int) -> Optional[ListNode]:
        # TODO: Implement your solution
        dummy = ListNode(0, head)
        prev = dummy
        end = head

        while True:
            cnt = 0
            temp = end
            while cnt < k and temp:
                temp = temp.next
                cnt += 1
            if cnt < k:
                break
            
            start = prev.next
            end = temp

            new = self.reverse(start, temp)
            prev.next = new  #链表链接

            prev = start #上一轮的边界自动变成prev，起到dummy的左右，下一轮的start就是prev.next
            end = prev.next # 起到head的作用

        return dummy.next

    # 这里的prev = end，是为了借用end起到prev.next = start的效果
    # end是开区间右边界，不参与反转，反转的是从 start 到 end 之前的所有节点
    # 从reverse这个函数可以看出，每一行代码之间的逻辑递进关系，每下一行都是受上一行变量影响而更新的
    def reverse(self, start, end):
        prev = end
        current = start
        while current != end:
            temp = current.next
            current.next = prev
            prev = current
            current = temp
        return prev

# Test cases
def test_reverseKGroup():
    s = Solution()
    head = build_linked_list([1,2,3,4,5])
    result = s.reverseKGroup(head, 2)
    assert linked_list_to_list(result) == [2,1,4,3,5]
    head = build_linked_list([1,2,3,4,5])
    result = s.reverseKGroup(head, 3)
    assert linked_list_to_list(result) == [3,2,1,4,5]
    print("All test cases passed!")

test_reverseKGroup()


All test cases passed!


## 138. Copy List with Random Pointer (Medium)

A linked list of length n is given such that each node contains an additional random pointer, 
which could point to any node in the list, or null.
Construct a deep copy of the list.

**Example:**
- Input: head = [[7,null],[13,0],[11,4],[10,2],[1,0]]
- Output: [[7,null],[13,0],[11,4],[10,2],[1,0]]


In [4]:
class Solution:
    def copyRandomList(self, head: Optional[RandomNode]) -> Optional[RandomNode]:
        # TODO: Implement your solution
        dummy = RandomNode(0, None, None)
        check = dict()

        # 先完成next指针的copy
        copy = dummy
        cur = head
        while cur:
            copy.next = RandomNode(cur.val, None, None)
            copy = copy.next
            check[cur] = copy # check[old_node] = new_node 这俩是一模一样的
            cur = cur.next

        # 再完成random指针
        copy = dummy.next
        cur = head
        while copy:
            copy.random = check.get(cur.random, None) #在这里，copy相当于原本的cur，那么copy.random的含义就是cur.random，那么其指针具体的指向就是check[cur.random]
            copy = copy.next
            cur = cur.next
        return dummy.next

# 真正深拷贝需要让新节点的next和random都指向新建的节点，而不是原有节点。
# Test cases
def test_copyRandomList():
    s = Solution()
    data = [(7, None), (13, 0), (11, 4), (10, 2), (1, 0)]
    head = build_random_list(data)
    result = s.copyRandomList(head)
    assert result is not None and result.val == 7
    assert s.copyRandomList(None) is None
    print("All test cases passed!")

test_copyRandomList()


All test cases passed!


## 146. LRU Cache (Medium)

Design a data structure that follows the constraints of a Least Recently Used (LRU) cache.

Implement the LRUCache class:
- LRUCache(int capacity) Initialize the LRU cache with positive size capacity.
- int get(int key) Return the value of the key if the key exists, otherwise return -1.
- void put(int key, int value) Update the value of the key if the key exists. 
  Otherwise, add the key-value pair to the cache.


In [5]:
# 需要一个双向链表子节点
class node:
    def __init__(self, key=0, val=0, prev=None, next=None) -> None:
        self.key = key
        self.val = val
        self.prev = prev
        self.next = next

class LRUCache:
    def __init__(self, capacity: int):
        # TODO: Implement your solution
        self.head = node()
        self.tail = node()
        self.head.next = self.tail
        self.tail.prev = self.head
        self.capacity = capacity
        self.map = dict()
    
    def __insert__(self, node:node):
        cur = self.head
        cur = cur.next
        node.prev = self.head
        node.next = cur
        cur.prev = node
        self.head.next = node

    def __remove__(self, node:node):
        if not node: return
        prev = node.prev
        next = node.next
        prev.next = next
        next.prev = prev

    def get(self, key: int) -> int:
        # TODO: Implement your solution
        if key not in self.map:
            return -1 
        target = self.map[key]
        self.__remove__(target)
        self.__insert__(target)
        return target.val

    def put(self, key: int, value: int) -> None:
        # TODO: Implement your solution
        if key not in self.map:
            new = node(key=key, val=value)
            self.__insert__(new)
            self.map[key] = new

            if self.capacity < len(self.map):
                cur = self.tail
                cur = cur.prev
                self.__remove__(cur)
                del self.map[cur.key]

        else:
            new = self.map[key]
            new.val = value
            self.__remove__(new)
            self.__insert__(new)

# 需支持“最近最少使用”淘汰策略，要在 O(1) 时间内把某个元素移到首/尾、删除最旧元素，双向链表可以 O(1) 完成这些操作。
# list 删除/插入中间元素是 O(n)，而双向链表能 O(1) 删除/插入节点，实现真正的 LRU 要求。
# 双向链表可以 O(1) 删除和插入任意节点（已知节点指针时）；单链表或 list 都无法同时高效完成这两个操作。

# 核心要点：hashtable + duallinked
# 每操作一次（无论get还是update）都要换一次位置
# 有key就更新并换位置，没有就插入并检查capacity，适当清空hashtable和linkedlist

# Test cases
def test_LRUCache():
    cache = LRUCache(2)
    cache.put(1, 1)
    cache.put(2, 2)
    assert cache.get(1) == 1
    cache.put(3, 3)
    assert cache.get(2) == -1
    cache.put(4, 4)
    assert cache.get(1) == -1
    assert cache.get(3) == 3
    assert cache.get(4) == 4
    print("All test cases passed!")

test_LRUCache()


All test cases passed!


## 📝 Notes

这三道题都好复杂啊，我感觉这里的medium比一般hard还复杂，不知道咋整的